In [1]:
import numpy as np
import pandas as pd 
import torch

import os
for dirname, _, filenames in os.walk('/Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning/Dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import pandas as pd
import re

/Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning/Dataset/test.csv
/Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning/Dataset/train.csv
/Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning/Dataset/sample_submission.csv


In [2]:
# Load the dataset into a pandas dataframe.
train_df = pd.read_csv('/Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning/Dataset/train.csv',encoding='UTF-8')

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(train_df.shape[0]))

# Display 10 random rows from the data.
train_df.sample(10)

Number of training sentences: 7,613



,id,keyword,location,text,target
5462,7790,police,Indonesia,Oops: Bounty hunters try to raid Phoenix polic...,1
7376,10559,windstorm,NaN,@charlesadler Ian Lee's word is like 'A fart i...,0
5491,7835,quarantine,NaN,Reddit Will Now Quarantine Offensive Content h...,0
5080,7244,natural%20disaster,"Littleton, CO",Is your team ready for a natural disaster a vi...,0
5551,7918,rainstorm,United States of America,'Three #people were #killed when a severe #rai...,1
5262,7523,oil%20spill,NaN,Refugio oil spill may have been costlier bigge...,1
7512,10745,wreckage,khanna,Wreckage 'Conclusively Confirmed' as From MH37...,1
7523,10759,wreckage,Southern California,Malaysian prime minister says Reunion Island w...,1
5180,7392,obliterate,Texas,Watch Sarah Palin OBLITERATE Planned Parenthoo...,0
7426,10623,wounded,NaN,Police Officer Wounded Suspect Dead After Exch...,1


In [3]:
emoji_pattern = re.compile("["         
         u"\U0001F600-\U0001F64F"  
         u"\U0001F300-\U0001F5FF"  
         u"\U0001F680-\U0001F6FF"  
         u"\U0001F1E0-\U0001F1FF"  
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [4]:
from nltk.tokenize import WordPunctTokenizer
import re
import emoji
from bs4 import BeautifulSoup
import itertools

tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'

def tweet_cleaner(text): 
   
    try:
        text1 = text.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        text1 = text
    
    #replace consecutive non-ASCII characters with a space
    text1 = re.sub(r'[^\x00-\x7F]+',' ', text1)
    
    #remove emojis from tweet
    text2 = emoji_pattern.sub(r'', text1)
    
    # HTML encoding
    soup = BeautifulSoup(text2, 'lxml') 
    text5 = soup.get_text()
    
    # removing @ mentions
    text6 = re.sub(pat1, '', text5)
    
    # Removing URLs
    text7 = re.sub(pat2, '', text6)

    # Fix misspelled words
    text9 = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text7))

    # Tokenizing ,change cases & join together to remove unneccessary white spaces
    text9_list = tok.tokenize(text9.lower())
    text10 = (" ".join(text9_list)).strip()
    
    return text10

# cleaning tweets
train_df['text_cleaned'] = list(map(lambda x:tweet_cleaner(x),train_df['text']) )

# checking out few samples
train_df.sample(10)

# Get the lists of sentences and their labels.
sentences = train_df.text_cleaned.values
labels = train_df.target.values

/Users/kaifrehmankhan/Library/Python/3.9/lib/python/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [5]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Print the original sentence.
print(' Original: ', sentences[0])

# Print the tweet split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the tweet mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 70,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[1])
print('Token IDs:', input_ids[1])

Loading BERT tokenizer...
 Original:  our deeds are the reason of this # earthquake may allah forgive us all
Tokenized:  ['our', 'deeds', 'are', 'the', 'reason', 'of', 'this', '#', 'earthquake', 'may', 'allah', 'forgive', 'us', 'all']
Token IDs:  [2256, 15616, 2024, 1996, 3114, 1997, 2023, 1001, 8372, 2089, 16455, 9641, 2149, 2035]


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Max sentence length:  67


/opt/homebrew/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  forest fire near la ronge sask . canada
Token IDs: tensor([  101,  3224,  2543,  2379,  2474,  6902,  3351, 21871,  2243,  1012,
         2710,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])


In [6]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

label_temp_list = []
for a,b,c in train_dataset:
  label_temp_list.append(c)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} training samples with real disater tweets'.format(sum(label_temp_list)))

label_temp_list = []
for a,b,c in val_dataset:
  label_temp_list.append(c)

print('{:>5,} validation samples'.format(val_size))
print('{:>5,} validation samples with real disater tweets'.format(sum(label_temp_list)))

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

6,851 training samples
2,933 training samples with real disater tweets
  762 validation samples
  338 validation samples with real disater tweets


In [7]:
from transformers import BertForSequenceClassification
from transformers import AdamW

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2, # The number of output labels--2 for binary classification.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [8]:
import random
import numpy as np
import time
import datetime
import torch
from transformers import AdamW, get_linear_schedule_with_warmup

def set_seed(seed_val=66):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def create_scheduler(optimizer, dataloader, epochs=2):
    total_steps = len(dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    return scheduler

def start_training(model, train_dataloader, valid_dataloader, optimizer, device='cpu', epochs=2):
    scheduler = create_scheduler(optimizer, train_dataloader, epochs)
    training_stats = train_and_evaluate(model, train_dataloader, valid_dataloader, optimizer, scheduler, epochs, device)
    return training_stats

def train_model_for_one_epoch(model, dataloader, optimizer, scheduler, device):
    total_train_loss = 0
    model.train()
    t0 = time.time()

    for step, batch in enumerate(dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f'Batch {step:>5} of {len(dataloader):>5}. Elapsed: {elapsed}.')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()    

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    avg_train_loss = total_train_loss / len(dataloader)
    training_time = format_time(time.time() - t0)

    return avg_train_loss, training_time

def evaluate_model(model, dataloader, device):
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    t0 = time.time()

    for batch in dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits

            total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(dataloader)
    avg_val_loss = total_eval_loss / len(dataloader)
    validation_time = format_time(time.time() - t0)

    return avg_val_accuracy, avg_val_loss, validation_time

def train_and_evaluate(model, train_dataloader, valid_dataloader, optimizer, scheduler, epochs=4, device='cpu'):
    set_seed()
    training_stats = []
    total_t0 = time.time()

    for epoch_i in range(0, epochs):
        print(f'======== Epoch {epoch_i+1} / {epochs} ========')
        print('Training...')

        t0 = time.time()

        avg_train_loss, training_time = train_model_for_one_epoch(model, train_dataloader, optimizer, scheduler, device)

        print(f"Average training loss: {avg_train_loss:.2f}")
        print(f"Training epoch took: {training_time}")

        print("Running Validation...")
        avg_val_accuracy, avg_val_loss, validation_time = evaluate_model(model, valid_dataloader, device)

        print(f"Accuracy: {avg_val_accuracy:.2f}")
        print(f"Validation Loss: {avg_val_loss:.2f}")
        print(f"Validation took: {validation_time}")

        training_stats.append({
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        })

    print("Training complete!")
    print(f"Total training took {format_time(time.time()-total_t0)} (h:mm:ss)")
    
    return training_stats

optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
training_stats = start_training(model, train_dataloader, validation_dataloader, optimizer, device='cpu', epochs=2)

======== Epoch 1 / 2 ========
Training...
Batch    40 of   215. Elapsed: 0:04:00.
Batch    80 of   215. Elapsed: 0:07:56.
Batch   120 of   215. Elapsed: 0:11:55.
Batch   160 of   215. Elapsed: 0:16:02.
Batch   200 of   215. Elapsed: 0:21:26.
Average training loss: 0.44
Training epoch took: 0:23:09
Running Validation...
Accuracy: 0.85
Validation Loss: 0.38
Validation took: 0:00:43
======== Epoch 2 / 2 ========
Training...
Batch    40 of   215. Elapsed: 0:03:52.
Batch    80 of   215. Elapsed: 0:07:42.
Batch   120 of   215. Elapsed: 0:11:39.
Batch   160 of   215. Elapsed: 0:15:31.
Batch   200 of   215. Elapsed: 0:19:18.
Average training loss: 0.33
Training epoch took: 0:20:46
Running Validation...
Accuracy: 0.86
Validation Loss: 0.37
Validation took: 0:00:39
Training complete!
Total training took 0:45:19 (h:mm:ss)


In [9]:
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

def load_and_clean_data(filepath, cleaner_function):
    # Load the dataset into a pandas dataframe.
    df = pd.read_csv(filepath)

    # Preprocess the text
    df['text_cleaned'] = df['text'].apply(cleaner_function)

    return df

def encode_sentences(sentences, tokenizer, max_length=75):
    input_ids = []
    attention_masks = []

    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
            sent,                        
            add_special_tokens=True,      
            max_length=max_length,           
            pad_to_max_length=True,
            return_attention_mask=True,   
            return_tensors='pt',         
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

def create_data_loader(input_ids, attention_masks, batch_size=32):
    data = TensorDataset(input_ids, attention_masks)
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

    return dataloader

df = load_and_clean_data('/Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning/Dataset/train.csv', tweet_cleaner)
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

sentences = df.text_cleaned.values
input_ids, attention_masks = encode_sentences(sentences, tokenizer)

prediction_dataloader = create_data_loader(input_ids, attention_masks)

/Users/kaifrehmankhan/Library/Python/3.9/lib/python/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Number of test sentences: 7,613



/opt/homebrew/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [10]:
import torch
import numpy as np

def predict_sentence(sentence, model, tokenizer, device):
    model.eval()  # Set the model to evaluation mode

    # Prepare the sentence
    encoded_dict = tokenizer.encode_plus(
        sentence,                      
        add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
        max_length=75,  # Pad or truncate
        pad_to_max_length=True,
        return_attention_mask=True,  # Construct attention masks
        return_tensors='pt',  # Return PyTorch tensors
    )
    
   
    b_input_ids = encoded_dict['input_ids'].to(device)
    b_input_mask = encoded_dict['attention_mask'].to(device)

    with torch.no_grad():
        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()

    pred_label = np.argmax(logits, axis=1)

    return pred_label

# Use the function
random_sentence = "Last year my mother died due to flood"
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Prediction time: ", predict_sentence(random_sentence, model, tokenizer, device))


Prediction time:  [1]


In [11]:
import os

output_dir = '/Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

model_to_save = model.module if hasattr(model, 'module') else model 
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to /Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning


('/Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning/tokenizer_config.json',
 '/Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning/special_tokens_map.json',
 '/Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning/vocab.txt',
 '/Users/kaifrehmankhan/Desktop/Algorithma/Bert_Finetuning/added_tokens.json')